In [1]:
import importlib
import keys
importlib.reload(keys)
import json
import pprint
keychain = keys.keychain

pp = pprint.PrettyPrinter(indent=2)

client_id = keychain['youtube']['client_id']
scope = keychain['youtube']['scope']
redirect = keychain['youtube']['redirect_uris'][0]
print(client_id[:20], scope, redirect)

126920892433-e58m7pn ['https://www.googleapis.com/auth/youtube.readonly'] urn:ietf:wg:oauth:2.0:oob


In [2]:
from requests_oauthlib import OAuth2Session

In [9]:
session = OAuth2Session(client_id, scope=scope, redirect_uri=redirect)
auth_url, state = session.authorization_url(keychain['youtube']['auth_uri'],access_type='offline',include_granted_scopes='true')
print(auth_url)

https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=126920892433-e58m7pn87600743drvlsjsanlcsus2av.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.readonly&state=5Qvcln7cMQyLfS0tA8eZPIbyM9tW8m&access_type=offline&include_granted_scopes=true


In [10]:
import webbrowser
webbrowser.open(auth_url)

True

In [11]:
code = '4/2tWn2YBuZHWtLfSZAVbhNXs6CulKrLbgxFpZvRmX8vs'

In [12]:
token = session.fetch_token(keychain['youtube']['token_uri'],
                            code=code,
                            client_secret=keychain['youtube']['client_secret'])  

In [13]:
print(token)

{'access_token': 'ya29.GlsaBUs7ZhJyYpfj1f2dXuLZ-wtlZ_XBYKUhFmoIx9TEZrKI0wRW5guhJX48jHSon_AJB-5_MTabRA9_64minpJJH-DhuReYAI0GRw_Z8F0BXZuLNp_6K6tMl0Jp', 'expires_in': 3599, 'refresh_token': '1/dtrPo6siitfCIqLcJvVU-utbXbKr_2r7FUswdAr4RtI', 'token_type': 'Bearer', 'expires_at': 1512600640.5067992}


In [ ]:
import json
with open('token.txt', 'w') as outfile:
    json.dump(token, outfile)

In [4]:
token=open('token.txt',mode='r')
token=token.read()
token

'{"access_token": "ya29.GlsaBbJyLGD0Ay68WjdquVzo6LUCO_8CyYY3apPjlKSK3Tl9SbYF7f3abTs_Wluv1VDOaA8N5A_D2-NM-_UgPKWdKBybF6HxnfSPgkXlyW2BK-yCV_Cis6LRfSJK", "expires_in": 3600, "refresh_token": "1/rqcoSbsJdHgVNWrvZaCq1FTu5GI15rf-DTFU0kZvjdc", "token_type": "Bearer", "expires_at": 1512600293.5420544}'

In [5]:
token=json.loads(token)
token

{'access_token': 'ya29.GlsaBbJyLGD0Ay68WjdquVzo6LUCO_8CyYY3apPjlKSK3Tl9SbYF7f3abTs_Wluv1VDOaA8N5A_D2-NM-_UgPKWdKBybF6HxnfSPgkXlyW2BK-yCV_Cis6LRfSJK',
 'expires_at': 1512600293.5420544,
 'expires_in': 3600,
 'refresh_token': '1/rqcoSbsJdHgVNWrvZaCq1FTu5GI15rf-DTFU0kZvjdc',
 'token_type': 'Bearer'}

In [6]:
importlib.reload(keys)
keychain = keys.keychain
keychain['youtube']

{'auth_provider_x509_cert_url': 'https://www.googleapis.com/oauth2/v1/certs',
 'auth_uri': 'https://accounts.google.com/o/oauth2/auth',
 'client_id': '126920892433-e58m7pn87600743drvlsjsanlcsus2av.apps.googleusercontent.com',
 'client_secret': 'flFfvozgUappY1MIRCPE5Cxc',
 'project_id': 'project5-187402',
 'redirect_uris': ['urn:ietf:wg:oauth:2.0:oob', 'http://localhost'],
 'scope': ['https://www.googleapis.com/auth/youtube.readonly'],
 'token_uri': 'https://accounts.google.com/o/oauth2/token'}

In [7]:
import requests
import pandas as pd
from xml.etree import ElementTree as ET
import io
D = {}
D['access_token'] = token['access_token']
url = 'https://www.googleapis.com/youtube/v3/channels?part=snippet&mine=true'
response = requests.get(url, params=D)
print(response)

<Response [200]>


In [8]:
url = 'https://www.googleapis.com/youtube/v3/videos'
D['regionCode'] = 'US'
D['chart'] = 'mostPopular'
D['part'] = ['snippet,contentDetails,statistics']
D['maxResults'] = '50'
popular_vid = requests.get(url, params = D)

popular_vid = popular_vid.json()
# popular_vid['nextPageToken']
popular_vid['nextPageToken']
D['pageToken'] = 'CDIQAA'
popular_vid = requests.get(url, params = D)
popular_vid = popular_vid.json()
popular_vid

{'etag': '"7991kDR-QPaa9r0pePmDjBEa2h8/6z6vFq3m78JJgup7sk4C0TSKGHg"',
 'items': [{'contentDetails': {'caption': 'false',
    'definition': 'hd',
    'dimension': '2d',
    'duration': 'PT1M43S',
    'licensedContent': True,
    'projection': 'rectangular'},
   'etag': '"7991kDR-QPaa9r0pePmDjBEa2h8/yypM-ePIbQiB5AJ6nsEOETB35Cc"',
   'id': 'w4lcP5ZA4wg',
   'kind': 'youtube#video',
   'snippet': {'categoryId': '15',
    'channelId': 'UCINb0wqPz-A0dV9nARjJlOQ',
    'channelTitle': 'The Dodo',
    'defaultAudioLanguage': 'en',
    'description': 'Dog Finds Family Who Loves All His Differences | Nobody wanted this dog because he looks different, but as soon as this couple saw him, they were SOLD.  They started noticing all the special things about him — like his very intense obsession with baths. For more of this special pup, you can check him out on Instagram: http://thedo.do/mrcornelius, and Facebook: https://www.facebook.com/themistercornelius/.\n\nLove Animals? Subscribe: https://www.you

In [ ]:
popular_vid['items']

data = {'id':[], 'categoryId':[], 'channelId':[], 'channelTitle':[], 'publishedAt':[],\
        'dislike':[],'like':[],'viewCount':[]}
for item in popular_vid['items']:
    data['id'].append(item['id'])
    data['categoryId'].append(item['snippet']['categoryId'])
    data['channelId'].append(item['snippet']['channelId'])
    data['channelTitle'].append(item['snippet']['channelTitle'])
    data['publishedAt'].append(item['snippet']['publishedAt'])
    data['dislike'].append(item['statistics']['dislikeCount'])
    data['like'].append(item['statistics']['likeCount'])
    data['viewCount'].append(item['statistics']['viewCount'])
df = pd.DataFrame(data)
df
# empty =[]
# for item in popular_vid['items']:
#     empty.append(item['statistics'])
# empty

In [ ]:
popular_vid


In [ ]:
refresh_url = keychain['youtube']['token_uri']
token = session.refresh_token(refresh_url, 
                             client_id=keychain['youtube']['client_id'],
                             client_secret=keychain['youtube']['client_secret'])

In [ ]:
print(token)